In [ ]:
import pandas as pd
import os

# work_dir = "../input/mnist-rotation"
work_dir = "../Task 3 Radiogram matching"
os.listdir(work_dir)

['sample_submission.csv',
 'submission.csv',
 'test_batches.csv',
 'test_conclusions.csv',
 'test_descriptions.csv',
 'train.csv']

In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast
from scipy.sparse import hstack

In [125]:
train = pd.read_csv(os.path.join(work_dir, 'train.csv'))
test_desc = pd.read_csv(os.path.join(work_dir, 'test_descriptions.csv'))
test_conc = pd.read_csv(os.path.join(work_dir, 'test_conclusions.csv'))
batches = pd.read_csv(os.path.join(work_dir, 'test_batches.csv'))

In [126]:
train.head(3)

,description,conclusion
0,На серии томограмм в малом тазу определяется с...,КТ-признаки асцита.
1,На серии КТ-сканов – лимфоузлы малого таза и п...,КТ-данных за вторичное поражение лимфатических...
2,На серии томограмм лимфоузлы малого таза не ув...,КТ-признаков вторичного поражения лимфоузлов и...


In [127]:
test_desc.head(3)

,description_id,description
0,desc_00001,На серии КТ-сканов - лимфоузлы малого таза и п...
1,desc_00002,На серии томограмм в 4 сегменте справа и 8 сег...
2,desc_00003,На серии КТ-сканнов – в паренхиме S10 сегмента...


In [128]:
test_conc.head(3)

,conclusion_id,conclusion
0,conc_00001,КТ-признаков вторичного поражения легких не вы...
1,conc_00002,КТ-данных за органические изменения головного ...
2,conc_00003,"КТ-картина кальцинатов, участков пневмофиброза..."


In [129]:
# text = pd.concat([train.description, train.conclusion, test_conc.conclusion, test_desc.description])

# vectorizer = TfidfVectorizer(
#     ngram_range=(1, 2),        # убираем частые слова
#     max_df=0.9,                 # убираем супервстречающиеся слова
# )
# vectorizer.fit(text)

In [130]:
# desc_vec = vectorizer.transform(test_desc['description'].fillna(""))
# conc_vec = vectorizer.transform(test_conc['conclusion'].fillna(""))

In [131]:
# desc_map = dict(zip(test_desc['description_id'], range(len(test_desc))))
# conc_map = dict(zip(test_conc['conclusion_id'], range(len(test_conc))))

In [132]:
# for (batch_id, desc_id), df_pair in batches.groupby(['batch_id', 'description_ids']):
#     print(desc_id)
#     break

In [133]:
vec_word = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1, 2),
    lowercase=True,
    max_df=0.7,
    min_df=2,
    sublinear_tf=True,
    norm='l2'
)
vec_char = TfidfVectorizer(
    analyzer="char_wb",      # границы слов → лучше мед-термины и коды
    ngram_range=(3, 5),
    lowercase=True,
    min_df=2,
    sublinear_tf=True,
    norm='l2'
)

corpus_desc = test_desc['description'].fillna("")
corpus_conc = test_conc['conclusion'].fillna("")
# можно fit на (train.description, train.conclusion, test_desc.description, test_conc.conclusion)
corpus_all = pd.concat([train['description'], train['conclusion'], corpus_desc, corpus_conc]).fillna("")

vec_word.fit(corpus_all)
vec_char.fit(corpus_all)

D = hstack([vec_word.transform(corpus_desc), vec_char.transform(corpus_desc)])
C = hstack([vec_word.transform(corpus_conc), vec_char.transform(corpus_conc)])

desc_map = dict(zip(test_desc['description_id'], range(len(test_desc))))
conc_map = dict(zip(test_conc['conclusion_id'], range(len(test_conc))))

def to_list(x):
    return x if isinstance(x, list) else ast.literal_eval(str(x))

preds = []
for _, g in batches.groupby('batch_id'):
    desc_ids = to_list(g['description_ids'].iloc[0])
    conc_ids = to_list(g['conclusion_ids'].iloc[0])

    conc_ids = [cid for cid in conc_ids if cid in conc_map]
    if not conc_ids:
        continue
    ci = [conc_map[cid] for cid in conc_ids]
    conc_mat = C[ci]   # кэшируем на батч

    for did in desc_ids:
        if did not in desc_map:
            continue
        di = desc_map[did]
        sims = cosine_similarity(D[di], conc_mat)[0]
        best_cid = conc_ids[int(np.argmax(sims))]
        preds.append((did, best_cid))

In [134]:
submission = pd.DataFrame(preds, columns=['description_id', 'conclusion_id'])
submission.to_csv('submission.csv', index=False)
submission

,description_id,conclusion_id
0,desc_00001,conc_00004
1,desc_00002,conc_00004
2,desc_00003,conc_00006
3,desc_00004,conc_00002
4,desc_00005,conc_00008
...,...,...
7365,desc_07366,conc_07366
7366,desc_07367,conc_07368
7367,desc_07368,conc_07367
7368,desc_07369,conc_07363
